#Classificação de Redes
##SME 0130 - Redes Complexas 1s/2021

#####Bruno Gazoni 7585037 <br> Bruno Baldissera Carlotto 10724351 <br> Luis Gustavo Peçanha 9806763 <br> Sergio Carrazzoni de Toledo Piza 9361073 <br> Rafael Ceneme 9898610


#Enunciado

Selecione 3 redes biológicas, 3 redes sociais e 3 redes tecnológicas desses endereços:

https://networks.skewed.de

http://konect.cc/networks/

https://icon.colorado.edu/


Faça a classificação das redes usando os modelos e medidas que aprendemos na aula.

Não se esqueça de selecionar o mesmo N e grau médio que a rede original na construção
dos modelos.

Verifique qual o modelo mais adequado para cada rede.

Hipótese: redes do mesmo tipo seguem o mesmo modelo.

Verifique se essa hipótese é verdadeira.

Bônus (não é obrigatório): identifique as principais diferenças entre os modelos. Isto é,
quais medidas mais contribuem para que uma rede seja classificada como sendo do
modelo BA.

#Introdução

Este trabalho busca classificar 9 redes de 3 tipos diferentes: 3 biológicas, 3 sociais e 3 tecnológicas, para analisar a recorrência das classificações de suas topologias. Em particular, estudaremos a possibilidade delas serem mais próximas dos modelos aleatórios de Erdos-Renyi, Small world, Barabasi-Albert ou Waxman random graph. A motivação para classificação em topologias é obter modelos de análise que podem ser manipulados para estudar possíveis ramificações futuras das redes e seus comportamentos em escalas ajustáveis para diversos experimentos, bem como procurar entender os princípios mais básicos estruturais que os diferentes tipos de redes naturais seguem.
#### Redes estudadas:
1. Yeast

  "Network of operons and their pairwise interactions, via transcription factor-based regulation, within the yeast Saccharomyces cerevisiae."

  Extraído de: https://networks.skewed.de/net/yeast_transcription


2. Malaria Genes

  "Networks of recombinant antigen genes from the human malaria parasite P. falciparum. Each of the 9 networks shares the same set of vertices but has different edges, corresponding to the 9 highly variable regions (HVRs) in the DBLa domain of the var protein. Nodes are var genes, and two genes are connected if they share a substring whose length is statistically significant. Metadata includes two types of node labels, both based on sequence structure around HVR6."

  Extraído de: https://networks.skewed.de/net/malaria_genes


3. Foodweb

  "Networks of carbon exchanges among species in the cypress wetlands of South Florida. One network covers the wet and the other the dry season. Each node represents a taxon (similar to a species), and a directed edge indicates that one taxon uses another as food."

  Extraído de: https://networks.skewed.de/net/foodweb_baywet

4. Facebook Friends

  "A small anonymized Facebook ego network, from April 2014. Nodes are Facebook profiles, and an edge exists if the two profiles are "friends" on Facebook. Metadata gives the social context for the relationship between ego and alter."

  Extraído de: https://networks.skewed.de/net/facebook_friends


5. Crime

  "A network of associations among suspects, victims, and/or witnesses involved in crimes in St. Louis in the 1990s. Data are derived from police records, via snowball sampling from five initial homicides. Left nodes are people, right nodes are crime events, and edges connect people to particular crimes events they were associated with. Metadata includes names, genders, and roles (suspects, victims, and/or witnesses)."

  Extraído de: https://networks.skewed.de/net/crime


6. Netscience

  "A coauthorship network among scientists working on network science, from 2006. This network is a one-mode projection from the bipartite graph of authors and their scientific publications."

  Extraído de: https://networks.skewed.de/net/netscience

7. Hens

  "Dominance interactions (pecking) among a group of White Leghorn hens, observed in 1946. Edge weights represent the frequency of interaction. The two versions below are equivalent."

  Extraído de: https://networks.skewed.de/net/hens


8. Student Cooperation

  "Network of cooperation among students in the "Computer and Network Security" course at Ben-Gurion University, in 2012. Nodes are students, and edges denote cooperation between students while doing their homework. The graph contains three types of links: Time, Computer, Partners."

  Extraído de: https://networks.skewed.de/net/student_cooperation


9. High Tech Company

  "Multiplex network of 3 edge types representing relationships (advice, friendship, and “reports to”) between managers of a high-tech company. Data hosted by Manlio De Domenico."

  Extraído de: https://networks.skewed.de/net/high_tech_company

Para a classificação, utilizaremos 16 medidas (a serem discutidas no código abaixo), variando de medidas sobre a distribuição dos graus dos vértices a medidas de centralidade, de densidade, dentre outras, que representarão cada rede num espaço amostral gerado por outras 120 redes aleatórias, 30 para cada tipo de classe citada acima. Faremos a classificação dentro deste espaço utilizando primeiro o método KNN e em seguida com Random Forests, neste segundo extraindo também a importância de cada medida na classificação resultante.

Partimos da hipótese de que redes de mesmo tipo possuem topologia mais semelhante. A seguir, montaremos o experimento, implementaremos os códigos, exporemos os resultados e enfim faremos a conclusão acerca da hipótese.

# Imports

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


# Functions

In [ ]:
def load_network(path):
  """
    Loads network given network path.

  args:
    network_path (String): Path to file containing network
  """

  G = nx.read_edgelist(path, delimiter=',', nodetype=int, data=(('weight',float),))
  G = G.to_undirected()
  # remove loops
  G.remove_edges_from(nx.selfloop_edges(G))
  # Se o grafo for desconectado, considera apenas o maior componente
  Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
  G = G.subgraph(Gcc[0])
  G = nx.convert_node_labels_to_integers(G, first_label=0)

  return G

In [ ]:
def visualize_network(G, i, title):
  """
    Shows visualization of network and prints the number of nodes and edges within it. 

  args:
    G (graph): Network to show.
    i: index for subplot position
    title: name of network to show above its ridiculogram 
  """

  '''N = len(G)
  M = G.number_of_edges()
  print('Number of nodes:', N)
  print('Number of edges:', M)'''

  pos = nx.spring_layout(G)
  plt.subplot(i); plt.title(title)
  nx.draw(G, pos, node_color="b", node_size=1, with_labels=False)

In [ ]:
def moment_of_degree_distribution(G, m):
  """
    Calculates moment of degree distribution for network G.

  args:
    G (graph): Network where moment of degree will be calculated.
    m (int):   The degree that will be calculated.  
  """
  M = 0
  N = len(G)
  for i in G.nodes:
        M = M + G.degree(i)**m
  M = M/N

  return M

In [ ]:
def shannon_entropy(G):
    """
        Calculates shannon entropy for network G.

        args:
            G (graph): Network where shannon entropy will be calculated.
    """
    k,Pk = degree_distribution(G)
    H = 0
    for p in Pk:
        if(p > 0):
            H = H - p*math.log(p, 2)
    return H

In [ ]:
def complexity_coef(G):
  """
    Calculates complexity coefficient for network G.

  args:
    G (graph): Network where complexity coefficient will be calculated.
  """

  cx = (moment_of_degree_distribution(G, 2) / moment_of_degree_distribution(G, 1))
  
  return cx

In [ ]:
def degree_distribution(G):
    vk = dict(G.degree())
    vk = list(vk.values()) # we get only the degree values
    maxk = np.max(vk)
    mink = np.min(min)
    kvalues= np.arange(0,maxk+1) # possible values of k
    Pk = np.zeros(maxk+1) # P(k)
    for k in vk:
        Pk[k] = Pk[k] + 1
    Pk = Pk/sum(Pk) # the sum of the elements of P(k) must to be equal to one
    return kvalues,Pk

In [ ]:
"""
      Extrai diversas medidas da rede G e as retorna num vetor. 
"""
def measures(G):
    measures = []

    # n de nos
    N = len(G)
    # n de arestas
    M = G.number_of_edges()

    # densidade
    den = N/M
    measures.append(den)

    vk = dict(G.degree())
    vk = list(vk.values())
    vk = np.array(vk)

    # diametro da rede
    d = nx.diameter(G)
    measures.append(d)

    # eficiencia global
    eg = nx.global_efficiency(G)
    measures.append(eg)

    # eficiencia local
    el = nx.local_efficiency(G)
    measures.append(el)

    # closeness centrality media
    CLC = dict(nx.closeness_centrality(G))
    CLC = list(CLC.values())
    av_clc = np.mean(CLC)
    measures.append(av_clc)

    # betweenness media
    B = dict(nx.betweenness_centrality(G))
    B = list(B.values())
    av_b = np.mean(B)
    measures.append(av_b)

    # pagerank medio
    PR = dict(nx.pagerank(G, alpha=0.85))
    PR = list(PR.values())
    av_pr = np.mean(PR)
    measures.append(av_pr)

    # 1o, 2o e 3o momentos de distribuição de grau
    k1 = moment_of_degree_distribution(G,1)
    k2 = moment_of_degree_distribution(G,2)
    k3 = moment_of_degree_distribution(G,3)
    measures.append(k1)
    measures.append(k2)
    measures.append(k3)

    # variancia da distribuição de grau
    var = k2 - k1**2
    measures.append(var)

    # clustering medio
    av_cl = nx.average_clustering(G)
    measures.append(av_cl)

    # menor caminho medio
    l = nx.average_shortest_path_length(G)
    measures.append(l)

    # coeficiente de assortatividade
    r = nx.degree_assortativity_coefficient(G)
    if math.isnan(r):
        r = 0
    measures.append(r)

    # entropia de shannon
    se = shannon_entropy(G)
    measures.append(se)

    # coeficiente de complexidade
    cx = complexity_coef(G)
    measures.append(cx)

    # transitividade
    t = nx.transitivity(G)
    measures.append(t)

    #den, d, eg, el, av_clc, k1, k2, k3, variance, av_cl, av_b, l, r, se, cx, t
    return measures

In [ ]:
# gerador de redes aleatórias

# Para cada rede natural do conjunto de teste chamamos essa função para gerar 30 redes de cada tipo (Erdos-Renyi, Small World, Barabasi-Albert e WG),
#  totalizando 120 redes, todas com o mesmo <k1> (primeiro momento) e número de vértices da rede natural recebida como parâmetro

def generate_trainset(n_nets, k1, N):
  count = 0
  cl = ['ER','WS','BA','WG']
  #ER networks (Erdos Renyi random graph)
  X = []
  y = []
  av_degree = k1
  p = av_degree/(N-1)

  for i in range(0,n_nets):
      GER = nx.gnp_random_graph(N, p, seed=None, directed=False)
      Gcc = sorted(nx.connected_components(GER), key=len, reverse=True)
      GER = GER.subgraph(Gcc[0])
      GER = nx.convert_node_labels_to_integers(GER, first_label=0)
      x = measures(GER)
      X.append(x)
      y.append(0.0)
      count += 1
      print(f"ER {i} ", end='')

  #WS networks (Watts and Strogatz small-world model)
  k = int(av_degree)
  p = 0.1 #probability of rewiring
  for i in range(0,n_nets):
      GWS = nx.watts_strogatz_graph(N, k, p, seed=None)
      Gcc = sorted(nx.connected_components(GWS), key=len, reverse=True)
      GWS = GWS.subgraph(Gcc[0])
      GWS = nx.convert_node_labels_to_integers(GWS, first_label=0)
      x = measures(GWS)
      X.append(x)
      y.append(1.0)
      count += 1
      print(f"WS {i} ", end='')

  # BA networks (Barabasi and Albert scale-free model)
  m = int(av_degree/2)
  for i in range(0,n_nets):
      GBA = nx.barabasi_albert_graph(N, m)
      Gcc = sorted(nx.connected_components(GBA), key=len, reverse=True)
      GBA = GBA.subgraph(Gcc[0])
      GBA = nx.convert_node_labels_to_integers(GBA, first_label=0)
      x = measures(GWS)
      X.append(x)
      y.append(2.0)
      count += 1
      print(f"BA {i} ", end='')

  # WG (Waxman random graph)
  for i in range(0, n_nets):
      GWG = nx.generators.geometric.waxman_graph(N)
      Gcc = sorted(nx.connected_components(GWG), key=len, reverse=True)
      GWG = GWG.subgraph(Gcc[0])
      x = measures(GWG)
      X.append(x)
      y.append(3.0)
      count += 1
      print(f"WG {i} ", end='')

  #matrizes de atributos
  X = np.array(X)
  y = np.array(y)

  print("")

  return X, y

In [ ]:
# Aqui fazemos a predição da topologia da rede conforme os 4 modelos de grafos aleatórios descritos,
#  usando para isso o vetor de medidas da rede, o modelo de ML usado e o scaler.

def predict(X_net, model, scaler):
  cl = ['ER','WS','BA','WG']

  X_net = np.array(X_net)
  X_net = X_net.reshape(1,len(X_net))
  # escala o vetor
  X_net = scaler.transform(X_net)

  y_pred = model.predict(X_net) 
  #print('Classe:', cl[int(y_pred)])
  
  return int(y_pred)

## ML Algorithms

In [ ]:
#normalização e chamada do KNN

def KNNclassifier(X, X_net, y, k=5):
  scaler = StandardScaler().fit(X)
  X = scaler.transform(X)

  X_net = np.array(X_net)
  X_net = X_net.reshape(1,len(X_net)) 
  X_net = scaler.transform(X_net)
  # print('Xnet:', X_net.shape)

  model = KNeighborsClassifier(n_neighbors=k, metric = 'euclidean')
  model.fit(X,y)
  return model, scaler
  # faz a predição no conjunto de teste

In [ ]:
# Faz a chamada do modelo Random Forest, a predição dada conforme sua execução e imprime-a, junto com o vetor de medidas em ordem
#  crescente de importância para a classificação.

def random_forest(X_net, X, y, name):
  #X_net = [den, d, eg, el, av_clc, k1, k2, k3, variance, av_cl, av_b, l, 0, se, cx, t]
  model, scaler = randomForestClassifier(X, X_net, y, 5)
  cl = ['ER','WS','BA','WG']

  # faz a predição
  prediction = predict(X_net, model, scaler)
  print('Classe random forest:', name, cl[int(prediction)])

  rAcclist = []

  importances = model.feature_importances_
  measurements_list = ["den", "d", "eg", "el", "av_clc", "k1", "k2", "k3", 
                      "variance", "av_cl", "av_b", "l", "r", "se", "cx", "t"]
  importances_list = []

  # obtém o vetor de importâncias
  for i in range(0, 16):
    importances_list.append([measurements_list[i], importances[i]])

  # ordena o vetor
  from operator import itemgetter
  print(sorted(importances_list, key=itemgetter(1)))

In [ ]:
# Chama o classificador Random Forest com o conjunto de treino X, vetor de features X_net e profundidade máxima maxDepth

def randomForestClassifier(X, X_net, y, maxDepth=4):
  y = y.astype(int)
  from sklearn.ensemble import RandomForestClassifier
  from sklearn.datasets import make_classification

  print(f"maxdepth {maxDepth} ", end='')

  scaler = StandardScaler().fit(X)
  X = scaler.transform(X)

  X_net = np.array(X_net)
  X_net = X_net.reshape(1,len(X_net)) 
  X_net = scaler.transform(X_net)

  clf = RandomForestClassifier(max_depth=maxDepth, random_state=0)
  clf.fit(X, y)
  return clf, scaler

In [ ]:
# Essa função auxiliar serve para chamar o classificador KNN a quantidade de vezes correta para as diferentes redes geradas, escolhendo também
#  o melhor k, otimizando o classificador. Também imprimimos a classificação dada pelo modelo para cada rede.

def apply_knn(X_net, X, y, name):  
  #print(f"o sheipe de xiszao eh {X.shape}")
  #substituí r por 0
  #X_net = [den, d, eg, el, av_clc, k1, k2, k3, variance, av_cl, av_b, l, 0, se, cx, t]
  y = y.astype(int)
  model, scaler = KNNclassifier(X, X_net, y, 4)

  #print(X_net)

  cl = ['ER','WS','BA','WG']

  prediction = predict(X_net, model, scaler)
  print('Classe knn:', name, cl[int(prediction)])

  #iteraremos por K diferentes para descobrir o óptimo
  acclist = []

  best_k = 0
  last_acc = 0.0

  for k in range (1, int(X.shape[0]/2)):
    classlist = []
    for i in range(0, X.shape[0]):
      currTrain = np.delete(X, i, 0)
      model, scaler = KNNclassifier(X, X_net, y, k)
      currclass = predict(X[i], model, scaler)
      classlist.append(currclass)
    #print(len(classlist))
    #print(y.shape)
    acc = accuracy_score(y, classlist)
    acclist.append(acc)
    if (acc > last_acc):
      best_k = k
    last_acc = acc

    print(f"sabemos:{y}")
    print(f"re-classificamos:{classlist}")

    print(f"acc para k igual a {k} é {acc}")

  model, scaler = KNNclassifier(X, X_net, y, best_k)
  prediction = predict(X_net, model, scaler)
  print('Classe knn (com melhor k):', name, cl[int(prediction)])

# Main

Loading networks for test.

### Biological

In [ ]:
# Preparamos o plot em uma mesma linha das 3 redes biológicas
plt.figure(figsize=(15,15))
b1 = load_network("/content/networks/bio/foodweb_baywet.csv")
visualize_network(b1, 331, "foodweb")
b2 = load_network("/content/networks/bio/malaria_genes.csv")
visualize_network(b2, 332, "malaria")
b3 = load_network("/content/networks/bio/yeast_transcription.csv")
visualize_network(b3, 333, "yeast")

### Social

In [ ]:
# Preparamos o plot em uma mesma linha das 3 redes sociais
s1 = load_network("/content/networks/social/crime.csv")
visualize_network(s1, 334, "crime")
s2 = load_network("/content/networks/social/facebook_friends.csv")
visualize_network(s2, 335, "fbfriends")
s3 = load_network("/content/networks/social/netscience.csv")
visualize_network(s3, 336, "netscience")

### Technological

In [ ]:
# Preparamos o plot em uma mesma linha das 3 redes tecnológicas
t1 = load_network("/content/networks/tech/hens.csv")
visualize_network(t1, 337, "hens")
t2 = load_network("/content/networks/tech/high_tech_company.csv")
visualize_network(t2, 338, "htechcompany")
t3 = load_network("/content/networks/tech/student_cooperation.csv")
visualize_network(t3, 339, "studentcoop")

Visualize networks.

In [ ]:
plt.show()

### Generate Measures

In [ ]:
# Essa célula az a chamada da função que gera os conjuntos de treino para cada rede, passando como parâmetro o número de nós da rede
#  e o primeiro momento de sua distribuição de grau, bem como o número de redes geradas de cada tipo (30).
b1_net = measures(b1)
b1X, b1Y = generate_trainset(30, b1_net[7], len(b1))
b2_net = measures(b2)
b2X, b2Y = generate_trainset(30, b2_net[7], len(b2))
b3_net = measures(b3)
b3X, b3Y = generate_trainset(30, b3_net[7], len(b3))

s1_net = measures(s1)
s1X, s1Y = generate_trainset(30, s1_net[7], len(s1))
s2_net = measures(s2)
s2X, s2Y = generate_trainset(30, s2_net[7], len(s2))
s3_net = measures(s3)
s3X, s3Y = generate_trainset(30, s3_net[7], len(s3))

t1_net = measures(t1)
t1X, t1Y = generate_trainset(30, t1_net[7], len(t1))
t2_net = measures(t2)
t2X, t2Y = generate_trainset(30, t2_net[7], len(t2))
t3_net = measures(t3)
t3X, t3Y = generate_trainset(30, t3_net[7], len(t3))

#print(X, y)

ER 0 ER 1 ER 2 ER 3 ER 4 ER 5 ER 6 ER 7 ER 8 ER 9 ER 10 ER 11 ER 12 ER 13 ER 14 ER 15 ER 16 ER 17 ER 18 ER 19 ER 20 ER 21 ER 22 ER 23 ER 24 ER 25 ER 26 ER 27 ER 28 ER 29 WS 0 WS 1 WS 2 WS 3 WS 4 WS 5 WS 6 WS 7 WS 8 WS 9 WS 10 WS 11 WS 12 WS 13 WS 14 WS 15 WS 16 WS 17 WS 18 WS 19 WS 20 WS 21 WS 22 WS 23 WS 24 WS 25 WS 26 WS 27 WS 28 WS 29 BA 0 BA 1 BA 2 BA 3 BA 4 BA 5 BA 6 BA 7 BA 8 BA 9 BA 10 BA 11 BA 12 BA 13 BA 14 BA 15 BA 16 BA 17 BA 18 BA 19 BA 20 BA 21 BA 22 BA 23 BA 24 BA 25 BA 26 BA 27 BA 28 BA 29 WG 0 WG 1 WG 2 WG 3 WG 4 WG 5 WG 6 WG 7 WG 8 WG 9 WG 10 WG 11 WG 12 WG 13 WG 14 WG 15 WG 16 WG 17 WG 18 WG 19 WG 20 WG 21 WG 22 WG 23 WG 24 WG 25 WG 26 WG 27 WG 28 WG 29 
ER 0 ER 1 ER 2 ER 3 ER 4 ER 5 ER 6 ER 7 ER 8 ER 9 ER 10 ER 11 ER 12 ER 13 ER 14 ER 15 ER 16 ER 17 ER 18 ER 19 ER 20 ER 21 ER 22 ER 23 ER 24 ER 25 ER 26 ER 27 ER 28 ER 29 WS 0 WS 1 WS 2 WS 3 WS 4 WS 5 WS 6 WS 7 WS 8 WS 9 WS 10 WS 11 WS 12 WS 13 WS 14 WS 15 WS 16 WS 17 WS 18 WS 19 WS 20 WS 21 WS 22 WS 23 WS 24 WS 25 WS 

/usr/local/lib/python3.7/dist-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: invalid value encountered in double_scalars
  return (xy * (M - ab)).sum() / numpy.sqrt(vara * varb)


ER 0 ER 1 ER 2 ER 3 ER 4 ER 5 ER 6 ER 7 ER 8 ER 9 ER 10 ER 11 ER 12 ER 13 ER 14 ER 15 ER 16 ER 17 ER 18 ER 19 ER 20 ER 21 ER 22 ER 23 ER 24 ER 25 ER 26 ER 27 ER 28 ER 29 WS 0 WS 1 WS 2 WS 3 WS 4 WS 5 WS 6 WS 7 WS 8 WS 9 WS 10 WS 11 WS 12 WS 13 WS 14 WS 15 WS 16 WS 17 WS 18 WS 19 WS 20 WS 21 WS 22 WS 23 WS 24 WS 25 WS 26 WS 27 WS 28 WS 29 BA 0 BA 1 BA 2 BA 3 BA 4 BA 5 BA 6 BA 7 BA 8 BA 9 BA 10 BA 11 BA 12 BA 13 BA 14 BA 15 BA 16 BA 17 BA 18 BA 19 BA 20 BA 21 BA 22 BA 23 BA 24 BA 25 BA 26 BA 27 BA 28 BA 29 WG 0 WG 1 WG 2 WG 3 WG 4 WG 5 WG 6 WG 7 WG 8 WG 9 WG 10 WG 11 WG 12 WG 13 WG 14 WG 15 WG 16 WG 17 WG 18 WG 19 WG 20 WG 21 WG 22 WG 23 WG 24 WG 25 WG 26 WG 27 WG 28 WG 29 
ER 0 ER 1 ER 2 ER 3 ER 4 ER 5 ER 6 ER 7 ER 8 ER 9 ER 10 ER 11 ER 12 ER 13 ER 14 ER 15 ER 16 ER 17 ER 18 ER 19 ER 20 ER 21 ER 22 ER 23 ER 24 ER 25 ER 26 ER 27 ER 28 ER 29 WS 0 WS 1 WS 2 WS 3 WS 4 WS 5 WS 6 WS 7 WS 8 WS 9 WS 10 WS 11 WS 12 WS 13 WS 14 WS 15 WS 16 WS 17 WS 18 WS 19 WS 20 WS 21 WS 22 WS 23 WS 24 WS 25 WS 

## Random Forest

In [ ]:
# Esta célula faz a chamada do Random forest para cada rede natural.

# bio
random_forest(b1_net, b1X, b1Y, "foodweb")
random_forest(b2_net, b2X, b2Y, "malaria")
random_forest(b3_net, b3X, b3Y, "yeast")

# social
random_forest(s1_net, s1X, s1Y, "crime")
random_forest(s2_net, s2X, s2Y, "fbfriends")
random_forest(s3_net, s3X, s3Y, "netscience")

# tech
random_forest(t1_net, t1X, t1Y, "hens")
random_forest(t2_net, t2X, t2Y, "htechcompany")
random_forest(t3_net, t3X, t3Y, "studentcoop")

print("###################################")

maxdepth 5 Classe random forest: foodweb ER
[['se', 0.014627680649568393], ['d', 0.016158187662767852], ['k3', 0.028122056996517986], ['den', 0.03196670187646398], ['k2', 0.035074699778989686], ['cx', 0.036368031903334104], ['variance', 0.04816458480003879], ['av_b', 0.050027738180299736], ['t', 0.056723629107358155], ['av_cl', 0.05812339493925672], ['l', 0.058437008721649364], ['av_clc', 0.09462559098937], ['r', 0.09805616133294112], ['el', 0.10245766997978442], ['k1', 0.10319180228300591], ['eg', 0.11069583545064009]]
maxdepth 5 Classe random forest: malaria ER
[['d', 0.01598011489613972], ['av_b', 0.03833720210130521], ['k3', 0.038526985256957864], ['k2', 0.03938850059539376], ['l', 0.04144742050472224], ['den', 0.04783517080483036], ['t', 0.04882482518119575], ['se', 0.04956017328342579], ['el', 0.061897277989975893], ['variance', 0.064704755104223], ['cx', 0.07333046683969265], ['av_cl', 0.07748398365531438], ['av_clc', 0.08508999596257623], ['k1', 0.08760441618641529], ['r', 0.08

## KNN

In [ ]:
# Fazemos a chamada da aplicação do KNN para cada rede natural

# bio
apply_knn(b1_net, b1X, b1Y, "foodweb")
apply_knn(b2_net, b2X, b2Y, "malaria")
apply_knn(b3_net, b3X, b3Y, "yeast")

# social
apply_knn(s1_net, s1X, s1Y, "crime")
apply_knn(s2_net, s2X, s2Y, "fbfriends")
apply_knn(s3_net, s3X, s3Y, "netscience")

# tech
apply_knn(t1_net, t1X, t1Y, "hens")
apply_knn(t2_net, t2X, t2Y, "htechcompany")
apply_knn(t3_net, t3X, t3Y, "studentcoop")

#Resultados e Discussão

##KNN

Implementamos um KNN em três etapas: em primeiro lugar, ele é treinado com um valor de k aleatório para estabelecer um valor de acurácia baseline. Em seguida, iterativamente variamos k entre 1 e até metade da quantidade de nós da rede e a acurácia é recalculada para cada k. A maior acurácia é identificada e utilizamos o valor de k associado a ela para treinar e classificar novamente a rede em questão, garantindo o melhor valor de k possível. Os resultados foram:

------------------------------------------------------

###Biológicas

####Foodweb: **Erdos-Renyi**

####Malaria: **Erdos-Renyi**

####Yeast: **Erdos-Renyi**

------------------------------------------------------

###Sociais

####Crime: **Erdos-Renyi**

####FBfriends: **Erdos-Renyi**

####Netscience: **Small world**

------------------------------------------------------

###Tecnológicas

####Hens: **Erdos-Renyi**

####Htechcompany: **Erdos-Renyi**

####Studentcoop: **Waxman random graph**

##Random forest

No algoritmo random forest, além da classificação também pudemos obter as features em ordem de importância. Abaixo listaremos as classes assinaladas para cada rede, as 3 features mais importantes e as 3 menos importantes de acordo com o classificador, arrendodadas e transformadas em porcentagem de importância. A lista completa de features ordenadas por importância em ordem crescente pode ser conferida na célula de retorno do random forest, acima.

Como comparação, se houvéssemos importância igual para cada feature o valor seria igual a 100%/16 = 6.25%.

## Biológicas
1. **Foodweb**:
##### Classificação: **Erdos-Renyi (ER)**

  ##### Top 3 features:
      * eficiência global: 11%;
      * primeiro momento: 10%;
      * eficiência local: 10%;

  ##### Bottom 3 features:
      * entropia de shannon: 1.4%;
      * diâmetro:  1.6%;
      * terceiro momento: 3%;

2. **Malaria**:
##### Classificação: **Erdos-Renyi (ER)**

  ##### Top 3 features:
    * eficiência global: 9.5%;
    * coeficiente de assortatividade: 8.7%;
    * primeiro momento: 8.7%

  ##### Bottom 3 features:
    * diâmetro: 1.6%
    * betweeness centrality: 3.8%
    * terceiro momento: 3.8%

3. **Yeast**:
##### Classificação: **Erdos-Renyi (ER)**

  ##### Top 3 features:
    * eficiência global: 8%
    * betweeness centrality: 7.8%
    * variância: 7.5%

  ##### Bottom 3 features:
    * segundo momento: 3%
    * entropia de shannon: 3.5%
    * primeiro momento: 3.6%

  ### Resultados para redes biológicas:
Todas foram classificadas como **Erdos-Renyi**. A feature mais importante para as 3 foi **eficiência global** e a mais frequente entre as menos significativas foi o diâmetro. Curiosamente, betweeness centrality ficou como uma das menos importantes para a rede malaria e uma das mais importantes para a rede yeast.

## Sociais

1. **Crime**:

  ##### Classificação: **Erdos-Renyi (ER)**
  ##### Top 3 features:
      * eficiência global: 13%;
      * assortatividade: 11%
      * closeness centrality: 10%

  ##### Bottom 3 features:
      * eficiência local: 2%;
      * shortest path length: 2.4%
      * primeiro momento: 4.2%

2. **Facebook Friends**:

  ##### Classificação: **Small world (WS)**

  ##### Top 3 features:
    * assortatividade: 14.4%
    * eficiência local: 14%
    * average clustering: 10%

  ##### Bottom 3 features:
    * segundo momento: 0%
    * densidade: 1%
    * diâmetro: 1.6%

3. **Netscience**:

  ##### Classificação: **Small world (WS)**
  ##### Top 3 features:
    * variância: 11%
    * coeficiente de complexidade: 9.6%
    * average clustering: 8.4%

  ##### Bottom 3 features:
    * segundo momento: 0.7%
    * densidade: 3%
    * diâmetro: 3.2%

  ### Resultados para redes sociais:
Duas foram classificadas como **Small World** e outra como **Erdos-Renyi**. A feature importante mais recorrente foi **average clustering**. As menos importantes mais recorrentes foram diâmetro, densidade e segundo momento.

## Tecnológicas

1. **Hens**:
  ##### Classificação: **Erdos-Renyi (ER)**

  ##### Top 3 features:
    * coeficiente de complexidade: 12%
    * caminho mais curto: 11%
    * average clustering: 10%

  ##### Bottom 3 features:
    * diâmetro: 1.8%
    * betweeness centrality: 2.1%
    * entropia de shannon: 2.6%

2. **High-Tech Company**:
  ##### Classificação: **Erdos-Renyi (ER)**

  ##### Top 3 features:
    * average clustering: 12.5%
    * variância: 9.2%
    * eficiência local: 8.5%

  ##### Bottom 3 features:
    * diâmetro: 0%
    * primeiro momento: 2%
    * assortatividade: 2%

3. **Student Cooperation Network**:
  ##### Classificação: **Waxman Random Graph (WG)**

  ##### Top 3 features:
    * primeiro momento: 11.3%
    * average clustering: 10%
    * coeficiente de complexidade: 9.7%

  ##### Bottom 3 features:
    * segundo momento: 1.2%
    * entropia de shannon: 1.3%
    * terceiro momento: 2.5%

  ### Resultados para redes tecnológicas:
  Duas foram classificadas como **Erdos-Renyi** e uma como **Waxman random graph**. A feature importante mais recorrente foi o **average clustering** e a feature menos importante mais recorrente foi a **entropia de Shannon**.


#Análise e conclusão

A motivação do experimento foi a hipótese: *redes de mesmo tipo possuem a mesma topologia*. Nossos resultados aplicaram análises em escalas relativamente pequenas porém obtivemos resultados que apontam para uma possível validação da hipótese. Redes biológicas tendem a ser Erdos-Renyi, sociais tendem a ser Small world e tecnológicas Erdos-Renyi.

O tamanho pequeno das amostras torna suas qualidades muito heterogêneas para interpretarmos conclusões em relação às qualidades de cada rede. Podemos entretanto ressaltar que era esperado, em particular, o comportamento Small world em redes sociais, dados seus average clustering e assortatividade elevados.




#Referências

1. [Ciência de Dados: k-vizinhos mais próximos (teoria e prática com Python)](https://www.youtube.com/watch?v=7WySJWL2o_4&t=169s&ab_channel=FranciscoRodrigues)

2. [Aula de Redes Complexas (dia 25/06/2021) -> a partir de 25:34](https://drive.google.com/file/d/1EVlXNXNYLI-WtD8ibk150aW1KdSkZ9-d/view)

3. ["A pattern recognition approach to complex networks"; L da F Costa, P R Villas Boas, F N Silva and F A Rodrigues](https://edisciplinas.usp.br/pluginfile.php/6308244/mod_resource/content/1/costa2010.pdf)